# Rewriting a reduced dataset

In [4]:
import pandas as pd
import os
import gc
import numpy as np

## Exploring the sample dataframe

In [1]:
import sys
from pathlib import Path

# Adding the project root for module imports
project_root = Path().resolve().parents[1]
sys.path.append(str(project_root))

from attendee_profiling import config

In [2]:
INPUT_PATH = config.INPUT_PATH
PROCESSED_PATH = config.PROCESSED_PATH

Assigning the path to read the files:

In [5]:
path_locations = os.path.join(INPUT_PATH, "wifi_traces_anonymized", "sonar_mod")

Assigning the path to write the files:

In [7]:
path_locations_reduced = os.path.join(PROCESSED_PATH, "wifi_traces_reduced")
print(f'Writing files in: "{os.path.abspath(path_locations_reduced)}"')

Writing files in: "C:\Camilo\Estudio\Padova\Master thesis\Datasets\Processed\wifi_traces_reduced"


Reading a sample file. To optimize operations when reading the complete set of files, I will use a dask dataframe, which operates lazily. To help querying the dataframe in the end, an initial sample file is read and its metadata (column names and types) is passed to the dask dataframe. 

In [3]:
# Load the single file into a pandas DataFrame
sample_df = pd.read_json(os.path.join(path_locations, 'anonymized_sonar_1718192117391157295_1718192602990134857.ndjson'),
                          lines=True)

# Visualizing some examples
sample_df.head(5)

,cell,cell_coordinates,client_coordinates,client_ipaddr,client_latitude,client_longitude,client_roll,client_username,client_xy_coordinates,confidence_factor,...,rssi,rssi_category,timestamp,visit_id,wdp_floor_id,zone_levels,zones_id,anonymized_macaddr,macaddr_randomized,vendor_name
0,4291204580560,"41.37343701985534,2.152411182412531","41.37357791098885,2.1525216172600397",10.93.131.17,41.373578,2.152522,NaN,None,"112,169.7",64.0,...,-62.0,poor,2024-06-12 11:35:07,visit-2878931198025351461,44bbfdef-41d8-421a-8775-b742a89f9136,[000Y.001E.001F],[f10d8100-ef6d-468b-acc4-09f8497cac7c],eb9a3a265d543758733d0631ff3298f02441b2bb1784bb...,1,None
1,4274804578820,"41.35761937221511,2.1330101228060645","41.35762750175618,2.1330771403851623",10.33.8.201,41.357628,2.133077,NaN,None,"333.2,232.2",40.0,...,-72.0,poor,2024-06-12 11:35:07,visit-6108473270290434404,11e594cf-47f5-47ea-9804-557e86ad7422,[],None,c71cffdc6b97dfee060558bf0bf381075b34ff0a9eb277...,1,None
2,4290404580540,"41.37324983687884,2.151456977505442","41.37335865635241,2.1516626585456895",10.93.131.38,41.373359,2.151663,NaN,None,"358.9,203.9",128.0,...,-87.0,poor,2024-06-12 11:35:07,visit-5034339734903319000,44bbfdef-41d8-421a-8775-b742a89f9136,[000Y.001E.001F],[f10d8100-ef6d-468b-acc4-09f8497cac7c],e2c4b71b708e80e715a2b6c725ca93bff81d28afce5943...,1,None
3,4278404578660,"41.356210648244144,2.137332548745098","41.35635670490924,2.1374764349553788",None,41.356357,2.137476,NaN,None,"101.5,88.2",32.0,...,-76.0,poor,2024-06-12 11:35:07,visit--1,bcb6b78f-200c-4bc8-93f2-fb06b4e0f31e,[],None,9955eaf4f0b351518f737625d6f9a815cb0120e51a5abe...,0,Intel Corporate
4,4275204578720,"41.35672230896424,2.1335002237455507","41.35675194450287,2.1337217515382854",10.33.18.6,41.356752,2.133722,NaN,None,"700,221.7",32.0,...,-67.0,poor,2024-06-12 11:35:07,visit-6964548436626391338,11e594cf-47f5-47ea-9804-557e86ad7422,[],None,926d4c8781e7242ad78682b94b2d1909df338782226ec9...,1,None


In [4]:
sample_df.dtypes

cell                              int64
cell_coordinates                 object
client_coordinates               object
client_ipaddr                    object
client_latitude                 float64
client_longitude                float64
client_roll                     float64
client_username                  object
client_xy_coordinates            object
confidence_factor               float64
device_id                        object
device_level                     object
event_source                     object
event_type                       object
floor_id                         object
floor_number                    float64
floorplans_id                    object
h3_cell                          object
h3_resolution                     int64
last_seen                         int64
layout_id                        object
location_id                      object
name                             object
parent                          float64
path                             object


In [5]:
type(sample_df['path'][0])

list

In [6]:
sample_df.shape

(100000, 35)

In [7]:
# Description of numerical columns
sample_df.describe()

,cell,client_latitude,client_longitude,client_roll,confidence_factor,floor_number,h3_resolution,last_seen,parent,rssi,timestamp,macaddr_randomized
count,1.000000e+05,100000.000000,100000.000000,0.0,98751.000000,44651.000000,100000.0,1.000000e+05,0.0,88180.000000,100000,100000.000000
mean,4.278820e+12,41.359959,2.137896,NaN,58.640419,1.352400,13.0,1.718192e+09,NaN,-66.590395,2024-06-12 11:39:06.771930112,0.712550
min,4.270205e+12,41.352517,2.127706,NaN,16.000000,-1.000000,13.0,1.718192e+09,NaN,-99.000000,2024-06-12 11:35:07,0.000000
25%,4.274805e+12,41.355878,2.133171,NaN,32.000000,1.000000,13.0,1.718192e+09,NaN,-74.000000,2024-06-12 11:36:58,0.000000
50%,4.277005e+12,41.356652,2.135812,NaN,48.000000,1.000000,13.0,1.718192e+09,NaN,-68.000000,2024-06-12 11:39:01,1.000000
75%,4.278405e+12,41.357601,2.137349,NaN,64.000000,2.000000,13.0,1.718192e+09,NaN,-61.000000,2024-06-12 11:41:12,1.000000
max,4.292605e+12,41.375206,2.154095,NaN,664.000000,3.000000,13.0,1.718193e+09,NaN,-15.000000,2024-06-12 11:43:22,1.000000
std,6.086304e+09,0.007179,0.007187,NaN,47.359395,0.756902,0.0,1.561842e+02,NaN,11.215159,NaN,0.452575


In [8]:
# Description of the non-numerical columns

# Get value counts for categorical columns
categorical_cols = ['event_source', 'event_type', 'floor_id', 'floorplans_id', 'h3_cell',
                    'layout_id', 'location_id', 'name', 'path', 'rssi_category', 'visit_id',
                    'wdp_floor_id', 'zone_levels', 'zones_id', 'anonymized_macaddr']

value_counts = {col: sample_df[col].value_counts() for col in categorical_cols}



# Print the value counts for each categorical column
for col, counts in value_counts.items():
    print(f"Value counts for column '{col}':\n{counts}\n")

Value counts for column 'event_source':
event_source
firehose    100000
Name: count, dtype: int64

Value counts for column 'event_type':
event_type
location    100000
Name: count, dtype: int64

Value counts for column 'floor_id':
floor_id
33250810-f622-4cf2-8d9a-f2c8e4aab278    26565
b33abe51-bb05-4f89-b4cb-5fc118865060    24946
4cc5e3bd-824b-46ed-9fdd-d5cd6177f4e8    12288
f5c592fe-a2e3-4ea2-a33d-d1b9da9a2b3b     7764
c5e72fa1-afca-4cb5-93cd-9612f9224b5c     4115
77e71e68-9af6-4348-9499-1d9e8c351210     2845
85b717ec-19ba-4d94-b4de-719389a34774     2482
0bacd607-4412-4ed6-96e3-43e794b4f44e     2269
1128741d-8d5e-41f8-a74b-e27f8a951a3c     1924
855aea98-26aa-4dbc-8d79-19e80d1ef37a     1735
692e863f-2c8e-438d-9179-2561d2288ba5     1672
a459e046-6a71-4f16-9b78-ee4810d9132c     1207
920edd56-5f1e-42c3-9254-435775377443     1164
5d36acae-f42a-46dc-b9d3-8cb745fc1345     1142
be5415cb-77c3-4cac-a697-fc7a4525ca40     1097
68d9ab9b-23e4-487c-97dc-b4388aceff3e     1078
64141cdf-8e19-40aa-b536-6

In [9]:
distinct_values_floor_name = sample_df.groupby(['name', 'floor_number'], dropna=False).size().reset_index(name='count').sort_values(by='count',ascending=False)
distinct_values_floor_name

,name,floor_number,count
13,Ground Floor,NaN,48725
12,First Floor,1.0,26565
21,Oficinas primera planta_v2,2.0,7764
22,Oficinas segunda planta_v2,3.0,4115
9,Espina 7 - 8,NaN,2482
19,Oficinas planta baja_v2,1.0,1735
8,Espina 57- 46,2.0,1207
23,Pl Univers,NaN,1164
10,Exhibition Level -1,-1.0,1142
3,Bajo Espino y CC7,NaN,1078


In [10]:
sample_df['zone_levels_as_str'] = sample_df['zone_levels'].astype(str)
sample_df['path_as_str'] = sample_df['path'].astype(str)
distinct_values_locid_name = sample_df.groupby(['name', 'location_id','zone_levels_as_str','path_as_str'], dropna=False).size().reset_index(name='count').sort_values(by=['name','location_id'],ascending=False)
distinct_values_locid_name

,name,location_id,zone_levels_as_str,path_as_str,count
53,avenia 3 4 DNA,location-84366ac4,['001A.001B.001C'],['0000.0002.0003.000g.001o.001A.001B.001C'],222
54,avenia 3 4 DNA,location-84366ac4,[],['0000.0002.0003.000g.001o.001A.'],14
50,Pl Univers,location-e3751b1b,['000Y.001E.001F'],['0000.0002.0003.000S.000X.000Y.001E.001F'],955
51,Pl Univers,location-e3751b1b,['000Y.001E.001G'],['0000.0002.0003.000S.000X.000Y.001E.001G'],116
52,Pl Univers,location-e3751b1b,[],['0000.0002.0003.000S.000X.000Y.'],93
49,Oficinas segunda planta_v2,location-4d326e20,[],['0000.0002.0003.000S.000U.001e.'],4115
44,Oficinas primera planta_v2,location-800f38e4,['001d.001o.001p'],['0000.0002.0003.000S.000U.001d.001o.001p'],1315
45,Oficinas primera planta_v2,location-800f38e4,['001d.001o.001q'],['0000.0002.0003.000S.000U.001d.001o.001q'],2153
46,Oficinas primera planta_v2,location-800f38e4,['001d.001o.001r'],['0000.0002.0003.000S.000U.001d.001o.001r'],1438
47,Oficinas primera planta_v2,location-800f38e4,['001d.001o.001s'],['0000.0002.0003.000S.000U.001d.001o.001s'],352


According to the information they gave me and from visual inspection, I think that the `slug` (a field needed later for joining the observations to their corresponding polygon) is obtained as the last part of the last string in the list contained in the field `path`. 

I try a function in the sample dataframe. Some observations end with ".", and I think that those do not correspond to the Sónar. I label those cases explicitly.


In [11]:
# Create DataFrame
sample_df

# Use numpy vectorized operation
def slug_from_path(path_column):
    # Convert the string representation of the list into actual lists
    path_list = np.array(path_column)
    # Extract the last item from each list
    last_strings = [lst[-1] if len(lst) > 0 else '' for lst in path_list]
    # Extract the last segment after the last dot in each string
    slug = ['ends_with_point' if s.endswith('.') else s.split('.')[-1] for s in last_strings]
    return slug

# Assign the new column
sample_df['slug'] = slug_from_path(sample_df['path'])
sample_df

,cell,cell_coordinates,client_coordinates,client_ipaddr,client_latitude,client_longitude,client_roll,client_username,client_xy_coordinates,confidence_factor,...,visit_id,wdp_floor_id,zone_levels,zones_id,anonymized_macaddr,macaddr_randomized,vendor_name,zone_levels_as_str,path_as_str,slug
0,4291204580560,"41.37343701985534,2.152411182412531","41.37357791098885,2.1525216172600397",10.93.131.17,41.373578,2.152522,NaN,None,"112,169.7",64.0,...,visit-2878931198025351461,44bbfdef-41d8-421a-8775-b742a89f9136,[000Y.001E.001F],[f10d8100-ef6d-468b-acc4-09f8497cac7c],eb9a3a265d543758733d0631ff3298f02441b2bb1784bb...,1,None,['000Y.001E.001F'],['0000.0002.0003.000S.000X.000Y.001E.001F'],001F
1,4274804578820,"41.35761937221511,2.1330101228060645","41.35762750175618,2.1330771403851623",10.33.8.201,41.357628,2.133077,NaN,None,"333.2,232.2",40.0,...,visit-6108473270290434404,11e594cf-47f5-47ea-9804-557e86ad7422,[],None,c71cffdc6b97dfee060558bf0bf381075b34ff0a9eb277...,1,None,[],['0000.0002.0003.000g.000s.000t.'],ends_with_point
2,4290404580540,"41.37324983687884,2.151456977505442","41.37335865635241,2.1516626585456895",10.93.131.38,41.373359,2.151663,NaN,None,"358.9,203.9",128.0,...,visit-5034339734903319000,44bbfdef-41d8-421a-8775-b742a89f9136,[000Y.001E.001F],[f10d8100-ef6d-468b-acc4-09f8497cac7c],e2c4b71b708e80e715a2b6c725ca93bff81d28afce5943...,1,None,['000Y.001E.001F'],['0000.0002.0003.000S.000X.000Y.001E.001F'],001F
3,4278404578660,"41.356210648244144,2.137332548745098","41.35635670490924,2.1374764349553788",None,41.356357,2.137476,NaN,None,"101.5,88.2",32.0,...,visit--1,bcb6b78f-200c-4bc8-93f2-fb06b4e0f31e,[],None,9955eaf4f0b351518f737625d6f9a815cb0120e51a5abe...,0,Intel Corporate,[],['0000.0002.0003.000g.000u.000z.'],ends_with_point
4,4275204578720,"41.35672230896424,2.1335002237455507","41.35675194450287,2.1337217515382854",10.33.18.6,41.356752,2.133722,NaN,None,"700,221.7",32.0,...,visit-6964548436626391338,11e594cf-47f5-47ea-9804-557e86ad7422,[],None,926d4c8781e7242ad78682b94b2d1909df338782226ec9...,1,None,[],['0000.0002.0003.000g.000s.000t.'],ends_with_point
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,4274004578740,"41.356891632058534,2.1320633860843365","41.356930522445715,2.1320890006895254",10.33.2.246,41.356931,2.132089,NaN,None,"319.6,165.9",32.0,...,visit-1092682443092512057,c6f13d53-164b-47a5-890e-8c7331389100,[],None,38f01950eb7f1e70aaf7e60cf9fccab569a9fec78a275f...,1,None,[],['0000.0002.0003.000g.000p.001i.'],ends_with_point
99996,4278004578620,"41.355846796683096,2.136859163441521","41.35588535760981,2.1369334627710184",10.33.3.199,41.355885,2.136933,NaN,None,"180.1,306.7",72.0,...,visit-9176399553934226916,bcb6b78f-200c-4bc8-93f2-fb06b4e0f31e,[],None,6a5db9067e60ffd4c4228a6bbee22162a16dce1e4d5a12...,1,None,[],['0000.0002.0003.000g.000u.000z.'],ends_with_point
99997,4278004578640,"41.35602692980924,2.1368567836020147","41.35610580291032,2.1370284031011755",10.33.16.149,41.356106,2.137028,NaN,None,"123.2,243.7",48.0,...,visit-8056765961774462430,bcb6b78f-200c-4bc8-93f2-fb06b4e0f31e,[],None,4adadd9a1b5bbc8df70dfe665fab892fbcbcbbd32c5c80...,1,None,[],['0000.0002.0003.000g.000u.000z.'],ends_with_point
99998,4277804578640,"41.35602513641843,2.1366177117913203","41.35609304227661,2.1367987355370723",10.33.22.201,41.356093,2.136799,NaN,None,"98.1,302.6",48.0,...,visit-8548297060232230374,bcb6b78f-200c-4bc8-93f2-fb06b4e0f31e,[],None,6eb277803d60a563baca11eda39f47be2ef3c81d12c07f...,1,None,[],['0000.0002.0003.000g.000u.000z.'],ends_with_point


In [12]:
# Removing the sample df to clear memory
del sample_df, categorical_cols, col, value_counts, counts
gc.collect()

0

# Sequentially rewriting a reduced and organized version of each file

To avoid operating with large files with unnecessary columns and to organize them before reading them with other tools, I rewrite a reduced version of the files (in csv).

In [14]:
# Specifying the desired column order
def rewrite_all_files(read_path, write_path, selected_columns):
    
    # Function to read, select and reorder columns for each file
    def reduce_rewrite_file(file_name, selected_columns):

        df = pd.read_json(os.path.join(read_path, file_name), lines=True, convert_dates=False)
        df['slug'] = slug_from_path(df['path']) # Obtaining the slug with the function defined above
        df = df[selected_columns] # Selection and reordering of the columns to match the desired order
        
        # Renaming columns to avoid confusions
        # According to what I was told, the timestamp column corresponds to
        # the time when the observation was received by the Wizzie system,
        # while last_seen corresponds to the time when the observation was
        # received by the access point (AP). I rename the columns accordingly.
        # timestamp_ap (last seen) is likely the one I need.
        df = df.rename(columns={'name':'location_name',
                                'timestamp':'timestamp_wizzie',
                                'last_seen':'timestamp_ap'})
        

        output_file_name = file_name.replace('anonymized_', 'reduced_').replace('.ndjson', '.csv')

        df.to_csv(os.path.join(write_path, output_file_name), index=False)


    # Iterating over the list files
    for i, file in enumerate(os.listdir(read_path)):

        # Executing the function
        reduce_rewrite_file(file_name=file, selected_columns=selected_columns)

        print(f'Rewrote file {i}')

    print('Finished rewriting the files.')

To avoid unnecessary memory overload, I first "filter" the dataframes, so that they contain only the possibly useful information. 
- With respect to the coordinates, after a manual check in a different file (in `Pruebas rápidas/Prueba coordenadas celdas h3.ipynb`), I concluded that using only the h3_cell is best thing.
- According to what I was told, `device_id` corresponds to the AP (access point), so only the (anonymized) macaddr is needed
- There are some [categorizations of RSSI](https://wiki.teltonika-networks.com/view/RSSI), which might diverge to the one that is stored.

In [15]:
desired_columns = ['anonymized_macaddr', 'macaddr_randomized',
                   'last_seen',
                   'h3_cell',
                   'client_latitude',
                   'client_longitude',
                   #'floor_number',
                   'timestamp', 
                   'name',# 'location_id',
                   'slug', # The variable obtained from 'path'
                   'rssi',
                   'confidence_factor',
                   'vendor_name']

In [16]:
rewrite_all_files(read_path=path_locations, 
                  write_path=path_locations_reduced,
                  selected_columns=desired_columns)

Rewrote file 0
Rewrote file 1
Rewrote file 2
Rewrote file 3
Rewrote file 4
Rewrote file 5
Rewrote file 6
Rewrote file 7
Rewrote file 8
Rewrote file 9
Rewrote file 10
Rewrote file 11
Rewrote file 12
Rewrote file 13
Rewrote file 14
Rewrote file 15
Rewrote file 16
Rewrote file 17
Rewrote file 18
Rewrote file 19
Rewrote file 20
Rewrote file 21
Rewrote file 22
Rewrote file 23
Rewrote file 24
Rewrote file 25
Rewrote file 26
Rewrote file 27
Rewrote file 28
Rewrote file 29
Rewrote file 30
Rewrote file 31
Rewrote file 32
Rewrote file 33
Rewrote file 34
Rewrote file 35
Rewrote file 36
Rewrote file 37
Rewrote file 38
Rewrote file 39
Rewrote file 40
Rewrote file 41
Rewrote file 42
Rewrote file 43
Rewrote file 44
Rewrote file 45
Rewrote file 46
Rewrote file 47
Rewrote file 48
Rewrote file 49
Rewrote file 50
Rewrote file 51
Rewrote file 52
Rewrote file 53
Rewrote file 54
Rewrote file 55
Rewrote file 56
Rewrote file 57
Rewrote file 58
Rewrote file 59
Rewrote file 60
Rewrote file 61
Rewrote file 62
Re